In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

In [3]:
from data_prep_functions import mnist_prep
from model_functions import *
from plotting_functions import *
import no_gpu
import reproducible

In [4]:
X_train, X_val, y_train, y_val = mnist_prep()

In [5]:
# use samller dataset for increased speed
X_train_small = X_train[:1000, :]
X_val_small = X_val[:500, :]
y_train_small = y_train[:1000]
y_val_small = y_val[:500]

n_cols = X_train_small.shape[1]

# EnKF

In [6]:
X_train = X_train_small
X_test = X_val_small
y_train = y_train_small
y_test = y_val_small

In [7]:
batch_size = 100
epochs = 5
particles = 10
early_stopping = 0.001
batch_normal = False
shuffle = True # noch einbauen!!!

In [8]:
layers = 5
neurons = [128, 128, 64, 32, 10]
n_cols = X_train.shape[1]

In [9]:
delta = 0.005
h_0 = 1
epsilon = 0.005

In [10]:
n = len(X_train)
num_batches = int(np.ceil(n / batch_size))
batch_indices = np.cumsum([0] + list(np.ones(num_batches) * batch_size))
batch_indices[-1] = n

In [11]:
X_batches = [X_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]
y_batches = [y_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]

In [12]:
model_dict = {}
weights_dict = {}
y_pred_dict = {}
jacobian_dict = {}
weights_vector_dict = {}

# init_model already has weights and biases following the Glorot distribution
# it can already be used to predict and evaluate, but it is very bad (<10% accuracy)
# only used to determine shapes and shape_elements via its weights
init_model = nn_model_structure(layers = layers,
                                neurons = neurons,
                                n_cols = n_cols)
init_model = nn_model_compile(init_model,
                              optimizer = "sgd")
weights = init_model.get_weights()
# shape contains the shapes of the weight matrices and bias vectors as a list of arrays
shapes = [np.array(params.shape) for params in weights]
# shape_elements contains the indices of the weights as a vector and tells where to cut
shape_elements = np.cumsum([0] + [np.prod(shape) for shape in shapes])

In [13]:
for i in range(1):
    # just an initial model with the correct structure regarding neurons, layers, activation functions, Glorot initialization
    model = nn_model_structure(layers = layers,
                               neurons = neurons,
                               n_cols = n_cols)
    model = nn_model_compile(model,
                             optimizer = "sgd")
    # for every particle write the model in a dictionary
    model_dict["model_{}".format(str(i+1))] = model
    
    # for every particles write the weights and biases in a dictionary
    weights_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                    .get_weights()

In [14]:
print(model_dict["model_1"].get_weights())

[array([[ 0.05316269,  0.01764145,  0.01522263, ...,  0.07493449,
        -0.0201476 ,  0.09051652],
       [-0.00516469,  0.07823637,  0.02811813, ..., -0.0174622 ,
        -0.02144132, -0.02854406],
       [-0.04033467,  0.00961152,  0.03930797, ..., -0.0174436 ,
         0.03845177, -0.03893438],
       ...,
       [ 0.04866969,  0.05048861, -0.01158144, ..., -0.03740012,
         0.01336047, -0.01347332],
       [ 0.01840364, -0.02679037,  0.03217081, ..., -0.0442641 ,
        -0.0488762 , -0.01088821],
       [-0.07488225, -0.07203993, -0.02490162, ...,  0.01187966,
         0.01317079,  0.01347652]], dtype=float32), array([-0.02369975,  0.033563  ,  0.06631573,  0.11554006, -0.05914802,
        0.08650133, -0.04362187,  0.11998139,  0.02238612,  0.05546513,
        0.07225065, -0.05853405, -0.02861994, -0.044171  ,  0.06349412,
       -0.18844545, -0.03606726, -0.01002328,  0.09155541,  0.18229206,
        0.20059158,  0.04867344, -0.15830076,  0.11646345, -0.02610004,
        0.

In [15]:
print(weights_dict["model_1"])

[array([[ 0.05316269,  0.01764145,  0.01522263, ...,  0.07493449,
        -0.0201476 ,  0.09051652],
       [-0.00516469,  0.07823637,  0.02811813, ..., -0.0174622 ,
        -0.02144132, -0.02854406],
       [-0.04033467,  0.00961152,  0.03930797, ..., -0.0174436 ,
         0.03845177, -0.03893438],
       ...,
       [ 0.04866969,  0.05048861, -0.01158144, ..., -0.03740012,
         0.01336047, -0.01347332],
       [ 0.01840364, -0.02679037,  0.03217081, ..., -0.0442641 ,
        -0.0488762 , -0.01088821],
       [-0.07488225, -0.07203993, -0.02490162, ...,  0.01187966,
         0.01317079,  0.01347652]], dtype=float32), array([-0.02369975,  0.033563  ,  0.06631573,  0.11554006, -0.05914802,
        0.08650133, -0.04362187,  0.11998139,  0.02238612,  0.05546513,
        0.07225065, -0.05853405, -0.02861994, -0.044171  ,  0.06349412,
       -0.18844545, -0.03606726, -0.01002328,  0.09155541,  0.18229206,
        0.20059158,  0.04867344, -0.15830076,  0.11646345, -0.02610004,
        0.

In [16]:
for i in range(1):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 563us/step - loss: 2.3359 - accuracy: 0.0900
0.09000000357627869


In [17]:
len(X_batches)

10

bei b=6 geht was schief -> vorher steigen die Accuracies (zumindest für manche Partikel), dann wieder die 8,4 %

da wird der Loss zu groß und dadurch kommen nan

da werden die Gewichte und Bias zu groß

kleineres $h_t$ hilft (Gewichte und Bias werden langsamer groß), aber Accuracy steigt weniger (vllt auch nur langsamer)

#### Lösungsidee: 1) Batch Normalization, 2) (muss sowieso auch noch gemacht werden) Mittelwerte der Partikel

### Nach dem ersten Schleifendurchlauf erhalten wir nicht mehr die Wahrscheinlichkeiten, sondern nur noch einen 0-1-Vektor.

In [18]:
for b in range(10):    
    for i in range(1):
        # for every particle write the predictions on the training batches in a dictionary
        y_pred_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                        .predict(X_batches[b])

        # for every particle write the Jacobian in a dictionary
        jacobian_dict["model_{}".format(str(i+1))] = (-1) * np.multiply(np.array(y_batches[b]), 
                                                                        np.array(1 / (y_pred_dict["model_{}".format(str(i+1))] + delta)))
    print(model_dict["model_1"].get_weights())
    print("-----------------------------------------------")
    print(y_pred_dict["model_1"])
    print("-----------------------------------------------")
    print("-----------------------------------------------")
    # bis hier hin alles gut     
    # compute the mean of the predictions
    y_pred_mean = np.mean(list(y_pred_dict.values()), axis = 0)
    # bis hier ok
    # compute the matrix D elementwise
    d = np.zeros(shape = (1, 1))
    for k in range(1):
        y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean + 0.001
        for j in range(1):
            d[k][j] = np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))
                                    # d sieht recht einfach aus, aber das wird wohl stimmen
    # compute the scalar h_t
    h_t = h_0 / (np.sqrt(np.sum(d**2)) + epsilon)
    # bis hier wohl ok
    # Reshape the weights and biases so that they are no longer matrices and vectores, but now one single vector
    for i in range(1):
        weights_array = np.array([])
        for j in range(len(weights_dict["model_{}".format(str(i+1))])):
            weights_array = np.append(weights_array, np.reshape(weights_dict["model_{}".format(str(i+1))][j], (1, -1)).ravel())
        weights_vector_dict["model_{}".format(str(i+1))] = weights_array
    # der Teil ist jetzt gedebugged    
    # matrix with particle parameters as row vectors
    weights_all_ptcls = np.array(list(weights_vector_dict.values()))

    # compute the matrix with the updates for each particle
    weights_all_ptcls = weights_all_ptcls - h_t * d * weights_all_ptcls

    for i in range(1):
        # write the updates back into the dictionary
        weights_vector_dict["model_{}".format(str(i+1))] = weights_all_ptcls[i]
        # reshape the updates, so that they are of the original matrx and vector shape
        for l in range(len(shape_elements)-1):
            start = shape_elements[l]
            end = shape_elements[l+1]
            weights_dict["model_{}".format(str(i+1))][l] = np.reshape(weights_vector_dict["model_{}".format(str(i+1))][start:end], tuple(shapes[l]))
        # set new weights for model
        model_dict["model_{}".format(str(i+1))].set_weights(weights_dict["model_{}".format(str(i+1))])

[array([[ 0.05316269,  0.01764145,  0.01522263, ...,  0.07493449,
        -0.0201476 ,  0.09051652],
       [-0.00516469,  0.07823637,  0.02811813, ..., -0.0174622 ,
        -0.02144132, -0.02854406],
       [-0.04033467,  0.00961152,  0.03930797, ..., -0.0174436 ,
         0.03845177, -0.03893438],
       ...,
       [ 0.04866969,  0.05048861, -0.01158144, ..., -0.03740012,
         0.01336047, -0.01347332],
       [ 0.01840364, -0.02679037,  0.03217081, ..., -0.0442641 ,
        -0.0488762 , -0.01088821],
       [-0.07488225, -0.07203993, -0.02490162, ...,  0.01187966,
         0.01317079,  0.01347652]], dtype=float32), array([-0.02369975,  0.033563  ,  0.06631573,  0.11554006, -0.05914802,
        0.08650133, -0.04362187,  0.11998139,  0.02238612,  0.05546513,
        0.07225065, -0.05853405, -0.02861994, -0.044171  ,  0.06349412,
       -0.18844545, -0.03606726, -0.01002328,  0.09155541,  0.18229206,
        0.20059158,  0.04867344, -0.15830076,  0.11646345, -0.02610004,
        0.

      dtype=float32)]
-----------------------------------------------
[[0.00000000e+00 1.34272870e-27 3.86253646e-07 0.00000000e+00
  7.93522783e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 9.92064416e-01]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.06395209e-12]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.20012841e-10]
 [0.00000000e+00 0.00000000e+00 1.22779375e-06 0.00000000e+00
  2.45528299e-22 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 9.99998808e-01]
 [0.00000000e+00 1.00713776e-19 2.77568655e-35 0.00000000e+00
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 5.84468036e-18]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00

      dtype=float32)]
-----------------------------------------------
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0

        -53.14544  ,  -35.70457  ], dtype=float32)]
-----------------------------------------------
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0

      dtype=float32)]
-----------------------------------------------
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0

In [19]:
for i in range(1):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 563us/step - loss: 144974658142208.0000 - accuracy: 0.1200
0.11999999731779099


In [20]:
model_dict

{'model_1': <tensorflow.python.keras.engine.sequential.Sequential at 0x182650262c8>}

In [21]:
model_dict["model_1"].get_weights()

[array([[ 54.23082  ,  17.995895 ,  15.528476 , ...,  76.440056 ,
         -20.552404 ,  92.33517  ],
        [ -5.268457 ,  79.80828  ,  28.683071 , ..., -17.813044 ,
         -21.87211  , -29.117565 ],
        [-41.14507  ,   9.804636 ,  40.097736 , ..., -17.794071 ,
          39.224335 , -39.716637 ],
        ...,
        [ 49.64755  ,  51.50302  , -11.814129 , ..., -38.151554 ,
          13.6289015, -13.744027 ],
        [ 18.773401 , -27.328638 ,  32.817184 , ..., -45.15345  ,
         -49.858215 , -11.106975 ],
        [-76.386765 , -73.48734  , -25.40194  , ...,  12.118345 ,
          13.43542  ,  13.74729  ]], dtype=float32),
 array([ -24.175922  ,   34.237335  ,   67.648125  ,  117.86147   ,
         -60.336407  ,   88.239296  ,  -44.498314  ,  122.39203   ,
          22.835892  ,   56.579525  ,   73.70229   ,  -59.710106  ,
         -29.194963  ,  -45.058475  ,   64.76983   , -192.23166   ,
         -36.79192   ,  -10.22467   ,   93.39492   ,  185.95464   ,
         204.62183

In [22]:
mean_weights = list(np.mean(list(weights_dict.values()), axis = 0))
init_model.set_weights(mean_weights)

In [23]:
init_model.evaluate(X_test, y_test)[1]

16/16 [==============================] - 0s 626us/step - loss: 144974658142208.0000 - accuracy: 0.1200


0.11999999731779099

In [24]:
y_pred_dict["model_1"]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 

In [25]:
jacobian_dict["model_1"]

array([[  -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        , -200.        ],
       [  -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        , -200.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        , -200.        ],
       [-200.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        , -200.        ,
          -0.        ,   -0.        ],
       [  -0.        , -200.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0